In [2]:
!pip install pydub
!pip install pyAudioAnalysis
!pip install hmmlearn
!pip install eyed3

     |████████████████████████████████| 52.4MB 94kB/s 
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.6-cp37-none-any.whl size=52589856 sha256=7f63839bd6057de76bab8cfa45582fc77ee7fc934d5d3b2c24c7bf77ff569681
  Stored in directory: /root/.cache/pip/wheels/fd/74/c2/361da76b03ed9d45c1b606d8fd25ac53ab965f754061fc4805
Successfully built pyAudioAnalysis
     |████████████████████████████████| 378kB 4.1MB/s 
     |████████████████████████████████| 256kB 11.3MB/s 
     |████████████████████████████████| 245kB 37.1MB/s 
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.5 which is incompatible.
  Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1


In [3]:
import os, sklearn.cluster
from pyAudioAnalysis.MidTermFeatures import mid_feature_extraction as mT
from pyAudioAnalysis.audioBasicIO import read_audio_file, stereo_to_mono
from pyAudioAnalysis.audioSegmentation import labels_to_segments
from pyAudioAnalysis.audioTrainTest import normalize_features
import numpy as np
import scipy.io.wavfile as wavfile
import IPython

def diarization_module(input_file):
  # read signal and get normalized segment feature statistics:
  fs, audio_data = read_audio_file(input_file)
  signal = stereo_to_mono(audio_data)
  mt_size, mt_step, st_win = 2, 0.1, 0.05
  [mt_feats, st_feats, _] = mT(audio_data, fs, mt_size * fs, mt_step * fs,
                            round(fs * st_win), round(fs * st_win * 0.5))
  (mt_feats_norm, MEAN, STD) = normalize_features([mt_feats.T])
  mt_feats_norm = mt_feats_norm[0].T

  # perform clustering
  n_clusters = 2
  x_clusters = [np.zeros((fs, )) for i in range(n_clusters)]
  k_means = sklearn.cluster.KMeans(n_clusters=n_clusters)
  k_means.fit(mt_feats_norm.T)
  cls = k_means.labels_

  # save clusters to concatenated wav files
  segs, c = labels_to_segments(cls, mt_step)  # convert flags to segment limits
  for sp in range(n_clusters):                
      count_cl = 0
      for i in range(len(c)):     # for each segment in each cluster (>2 secs long)
          if c[i] == sp and segs[i, 1]-segs[i, 0] > 2:
              count_cl += 1
              # get the signal and append it to the cluster's signal (followed by some silence)
              cur_x = audio_data[int(segs[i, 0] * fs): int(segs[i, 1] * fs)]
              x_clusters[sp] = np.append(x_clusters[sp], cur_x)
              x_clusters[sp] = np.append(x_clusters[sp], np.zeros((fs,)))
      # write cluster's signal into a WAV file
      print(f'speaker {sp}: {count_cl} segments {len(x_clusters[sp])/float(fs)} sec total duration')        
      wavfile.write(f'diarization_cluster_{sp}.wav', 16000, np.int16(x_clusters[sp]))
      IPython.display.display(IPython.display.Audio(f'diarization_cluster_{sp}.wav'))

input_file = "/content/silence_free.wav"
diarization_module(input_file)

speaker 0: 2 segments 21.6 sec total duration


speaker 1: 2 segments 18.9 sec total duration
